<a href="https://colab.research.google.com/github/Chieh-YuHung-hub/HTM737/blob/main/Week_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the dataset**

In [1]:
import pandas as pd

# Load ED data
df = pd.read_csv('https://www.dropbox.com/scl/fi/of5glv327d7wrgstpe8fc/df_ed.csv?rlkey=0m58ktluzuf6d439zjnv035da&dl=1')

print(f"Dataset: {df.shape[0]} rows/observations, {df.shape[1]} columns/fields")
df.head()

/tmp/ipython-input-4282020905.py:4: DtypeWarning: Columns (52,191,513,533,553) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('https://www.dropbox.com/scl/fi/of5glv327d7wrgstpe8fc/df_ed.csv?rlkey=0m58ktluzuf6d439zjnv035da&dl=1')


Dataset: 24777 rows/observations, 579 columns/fields


,VMONTH,VDAYR,ARRTIME,WAITTIME,LOV,AGE,AGER,AGEDAYS,RESIDNCE,SEX,...,RX12V3C1,RX12V3C2,RX12V3C3,RX12V3C4,SETTYPE,YEAR,CSTRATM,CPSUM,PATWT,EDWT
0,1,3,647,33,58,46,4,-7,1,2,...,NaN,NaN,NaN,NaN,3,2013,20113201,100020,2945,NaN
1,1,3,1841,109,150,56,4,-7,1,2,...,NaN,NaN,NaN,NaN,3,2013,20113201,100020,2945,NaN
2,1,3,1333,84,198,37,3,-7,1,2,...,NaN,NaN,NaN,NaN,3,2013,20113201,100020,2945,NaN
3,1,3,1401,159,276,7,1,-7,1,1,...,NaN,NaN,NaN,NaN,3,2013,20113201,100020,2945,NaN
4,1,4,1947,114,248,53,4,-7,1,1,...,NaN,NaN,NaN,NaN,3,2013,20113201,100020,2945,NaN


**Creating the target variable**

In [3]:
# These columns indicate different types of admission
response_cols = ['ADMITHOS','TRANOTH','TRANPSYC','OBSHOS','OBSDIS']

# If ANY admission, set to 1
df['ADMITFINAL'] = (df[response_cols].sum(axis=1) >= 1).astype(int)

# Drop original columns
df = df.drop(response_cols, axis=1)

print("Admission rate:")
print(df['ADMITFINAL'].value_counts())
print(f"\n{df['ADMITFINAL'].mean():.1%} were admitted")

Admission rate:
ADMITFINAL
0    21358
1     3419
Name: count, dtype: int64

13.8% were admitted


In [4]:

df['ADMITFINAL'].head()

,ADMITFINAL
0,0
1,0
2,0
3,0
4,1


In [5]:
# See what we have
print("WAITTIME distribution:")
print(df['WAITTIME'].value_counts().head(20))
#What values are invalid here?

WAITTIME distribution:
WAITTIME
-9     3635
-7      673
 0      673
 10     513
 8      511
 6      509
 5      505
 7      495
 4      465
 9      458
 12     442
 15     439
 11     435
 13     428
 17     416
 16     414
 14     411
 1      404
 3      401
 2      386
Name: count, dtype: int64


In [9]:
# Fix: Replace -7 and -9 with NaN
df['WAITTIME'] = df['WAITTIME'].replace([-7, -9], None)

print("After fixing:")
print(df['WAITTIME'].describe())
print(f"\nMissing: {df['WAITTIME'].isnull().sum()}")

After fixing:
count    24777.000000
mean        45.868386
std         67.135120
min          0.000000
25%         12.000000
50%         32.000000
75%         45.868386
max       1227.000000
Name: WAITTIME, dtype: float64

Missing: 0


**Missing values imputation**

In [10]:
# Fill missing wait times with average
mean_wait = df['WAITTIME'].mean()
print(f"Average wait time: {mean_wait:.1f} minutes")

df['WAITTIME'] = df['WAITTIME'].fillna(mean_wait)

print(f"Missing after: {df['WAITTIME'].isnull().sum()}")

#Key functions here are ... and ...

Average wait time: 45.9 minutes
Missing after: 0


In [11]:
df['WAITTIME'] = df['WAITTIME'].fillna(mean_wait).infer_objects(copy=False)

B- Mode imputation (for categorical variables) Mode = the most repeated/common value

In [12]:
# Fill missing SEX with most common value
print("SEX distribution:")
print(df['SEX'].value_counts())

mode_sex = df['SEX'].mode()[0]
df['SEX'] = df['SEX'].fillna(mode_sex)
#print(mode_sex)

SEX distribution:
SEX
1    13702
2    11075
Name: count, dtype: int64


**Feature Engineering**

Create a new feature using existing data

In [13]:
df['WINTER'] = df['VMONTH'].isin([12, 1, 2, 3]).astype(int)

print("Winter visits:")
print(df['WINTER'].value_counts())

Winter visits:
WINTER
0    16584
1     8193
Name: count, dtype: int64


In [14]:
df['NIGHT'] = ((df['ARRTIME'] < 800) | (df['ARRTIME'] >= 2000)).astype(int)

print(f"{df['NIGHT'].mean():.1%} arrive at night")

33.8% arrive at night


**Drop unnecessary columns**

In [15]:
# Drop redundant time columns (we created WINTER, NIGHT)
df = df.drop(['VMONTH', 'ARRTIME', 'VDAYR'], axis=1)

# Drop identifiers
df = df.drop(['PATCODE', 'HOSPCODE'], axis=1)

# Drop AGEDAYS (we have AGE in years)
df = df.drop(['AGEDAYS'], axis=1)

print(f"Columns remaining: {df.shape[1]}")

#This could be for columns that aren't available at the time of the prediction e.g., at the start of the ER visit.

Columns remaining: 571
